<a href="https://colab.research.google.com/github/Robert-Gomez-AI/HeartFailure/blob/main/M2U5_fase_4_Modelamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1A9dhO0gYByNJf9ya0SZmeCzZIeepRhsO" alt = "Encabezado MLDS" width = "100%">  </img>

# **Modelamiento y Validación**
---

Este notebook es una plantilla que le puede servir como guía para el cuarto entregable del proyecto aplicado.

## **0. Integrantes del equipo de trabajo**
---

Por favor incluya los nombres completos y número de identificación de los integrantes del equipo de trabajo:

1. ---**INGRESE SU RESPUESTA**---
2. ---**INGRESE SU RESPUESTA**---
3. ---**INGRESE SU RESPUESTA**---

## **1. Selección y diseño de modelos**
---

Seleccione el/los modelo/s a explorar para aplicar sobre el conjunto de datos. Recuerde que la selección del modelo está influenciada por diferentes factores.

Además, debe definir cuál es su problema:

- **Regresión**: se busca estimar un valor continúo a partir de los datos.
- **Clasificación**: permite estimar un valor categórico a partir de los datos.
- **Agrupamiento**: permite encontrar grupos de datos similares.
- **Otros modelos**: recuerde que dispone de otros tipos de tareas supervisadas y no supervisadas.

Justifique la selección y el diseño de los modelos a explorar.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
np.random.seed(42)



# Validation and searching utils
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report




import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df= pd.read_csv('/content/heart_failure_clinical_records_dataset.csv')

In [4]:
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


In [6]:
print("Class Percentage:")
df["DEATH_EVENT"].value_counts(normalize=True)

Class Percentage:


DEATH_EVENT
0    0.67893
1    0.32107
Name: proportion, dtype: float64

In [7]:
X =df.drop('DEATH_EVENT',axis=1).values
y= df['DEATH_EVENT'].values

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

---**INGRESE SU RESPUESTA**---

## **2. Entrenamiento del Modelo y Selección de Hiperparámetros**
---

Entrene el modelo y consiga la combinación eficiente de hiperparámetros para obtener un rendimiento óptimo.

Es importante visualizar los resultados de manera clara y comprensible. Para esto, **se deben utilizar tablas y gráficos** que muestren cómo cambian las métricas de desempeño del proceso de entrenamiento en función de los hiperparámetros probados. Algunas herramientas visuales útiles incluyen las *curvas de aprendizaje* y las *curvas de validación,* que nos permiten comparar el rendimiento del modelo con diferentes ajustes de hiperparámetros.

## Support vector machine

In [8]:


hyperparameters = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': [0.1, 1, 10]
}


model = SVC()

# Create the GridSearchCV object with the model and hyperparameters
grid_search = GridSearchCV(model, hyperparameters, cv=5)

# Fit the model using GridSearchCV and perform cross-validation
grid_search.fit(X, y)



GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10],
                         'kernel': ['linear', 'rbf']})

In [9]:
# Get the best found hyperparameters
best_params = grid_search.best_params_

#  Create a new model with the best hyperparameters
svm = SVC(**best_params)

# Compute the cross-validation scores using the best model
cross_val_scores = cross_val_score(svm, X, y, cv=5)

# Print the average cross-validation score
print("Average cross-validation score:", cross_val_scores.mean())

#  Train the model using all data and the best hyperparameters
svm.fit(X, y)

Average cross-validation score: 0.7822598870056497


SVC(C=10, gamma=0.1, kernel='linear')

## Logistic regression

In [10]:
hyperparameters = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000,10000] ,
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}


model = LogisticRegression()

# Create the GridSearchCV object with the model and hyperparameters
grid_search = GridSearchCV(model, hyperparameters, cv=5)

# Fit the model using GridSearchCV and perform cross-validation
grid_search.fit(X, y)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [11]:
# Get the best found hyperparameters
best_params = grid_search.best_params_

#  Create a new model with the best hyperparameters
Lg = LogisticRegression(**best_params)

#  Compute the cross-validation scores using the best model
cross_val_scores = cross_val_score(Lg, X, y, cv=5)

#  Print the average cross-validation score
print("Average cross-validation score:", cross_val_scores.mean())

#  Train the model using all data and the best hyperparameters
Lg.fit(X, y)

Average cross-validation score: 0.7889265536723163


LogisticRegression(C=1, penalty='l1', solver='liblinear')

In [12]:
hyperparameters = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


model = DecisionTreeClassifier()


grid_search = GridSearchCV(model, hyperparameters, cv=5)


grid_search.fit(X, y)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [13]:
best_params = grid_search.best_params_


D3 = DecisionTreeClassifier(**best_params)


cross_val_scores = cross_val_score(D3, X, y, cv=5)


print("Average cross-validation score:", cross_val_scores.mean())


D3.fit(X, y)

Average cross-validation score: 0.6689265536723163


DecisionTreeClassifier(min_samples_leaf=4)

In [15]:


hyperparameters = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

model = KNeighborsClassifier()


grid_search = GridSearchCV(model, hyperparameters, cv=5)


grid_search.fit(X, y)



GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7],
                         'weights': ['uniform', 'distance']})

In [16]:


best_params = grid_search.best_params_


Knn = KNeighborsClassifier(**best_params)



print("Average cross-validation score:", cross_val_scores.mean())


Knn.fit(X, y)



Average cross-validation score: 0.6689265536723163


KNeighborsClassifier(metric='euclidean', n_neighbors=3)

## **3. Evaluación del modelo**
---

En esta sección debe reportar el desempeño del modelo sobre la partición de datos de pruebas (test). Considere que dispone de las siguientes métricas:

- **Clasificación**: accuracy, precision, recall, f1-score, AUC, matriz de confusión, etc.
- **Regresión**: $r^2$, error cuadrático medio, error absoluto medio, etc.
- **Agrupamiento**: coeficiente de silueta, índice de Davies-Bouldin, etc.

Los resultados se deben presentar usando tablas y figuras además del análisis detallado respectivo.

In [17]:
def evaluate_model(model, X, y):

    y_pred = model.predict(X)

    metrics = {
        'Accuracy': accuracy_score(y, y_pred),
        'Precision': precision_score(y, y_pred),
        'Recall': recall_score(y, y_pred),
        'F1-Score': f1_score(y, y_pred)
    }

    return metrics


In [18]:
models = [svm,Knn,Lg, D3]
model_type=['Support Vector Machine','K-Nearest Neighbors','Logistic Regression','Decision three']
count=0
for model in models:

    print(model_type[count])
    evaluation=evaluate_model(model,X,y)
    for key in evaluation:
        print (f'{key}:{evaluation[key]}')
    count+=1
    print('\n')


Support Vector Machine
Accuracy:0.8494983277591973
Precision:0.7931034482758621
Recall:0.71875
F1-Score:0.7540983606557378


K-Nearest Neighbors
Accuracy:0.8294314381270903
Precision:0.8813559322033898
Recall:0.5416666666666666
F1-Score:0.6709677419354838


Logistic Regression
Accuracy:0.842809364548495
Precision:0.7951807228915663
Recall:0.6875
F1-Score:0.7374301675977654


Decision three
Accuracy:0.9264214046822743
Precision:0.9512195121951219
Recall:0.8125
F1-Score:0.8764044943820225




In [ ]:
# ---**INGRESE SU CÓDIGO**---

Adicionalmente, si utilizó un modelo pre-entrenado, debe generar predicciones y mostrar el resultado final de la aplicación del modelo sobre sus datos:

In [ ]:
# ---**INGRESE SU CÓDIGO**---

## **4. Aplicación del modelo**
---

Una vez que haya entrenado y evaluado su modelo, es importante considerar cómo puede ser utilizado en un entorno práctico.

Intente analizar la capacidad del modelo para generalizar nuevos datos que no formaron parte del conjunto de entrenamiento o evaluación.

In [ ]:
# ---**INGRESE SU CÓDIGO**---

## **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistente docente :**
  * [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandrasuperlano/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](https://www.linkedin.com/in/alejandrasuperlano/).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*